In [11]:
"""
Knowledge-Based Matchmaking System
---------------------------------
This system matches user profiles based on weighted compatibility scores across multiple attributes.
The KBS maintains profiles and uses domain-specific rules (weights, similarity metrics) to determine matches.
"""

class KnowledgeBase:
    """
    Stores and manages user profiles with unique identifiers.
    Acts as the knowledge repository for the matchmaking system.
    """
    def _init_(self):
        self.profiles = {}  # {profile_id: profile_data}
        self.next_id = 1     # Auto-incrementing ID counter

    def add_profile(self, profile_data):
        """
        Adds a new profile to the knowledge base with automatic ID assignment
        :param profile_data: Dictionary containing profile attributes
        :return: Assigned profile ID
        """
        profile_id = self.next_id
        self.profiles[profile_id] = {'id': profile_id, **profile_data}
        self.next_id += 1
        return profile_id

    def remove_profile(self, profile_id):
        """Removes a profile from the knowledge base"""
        if profile_id in self.profiles:
            del self.profiles[profile_id]

    def get_profile(self, profile_id):
        """Retrieves a profile by ID"""
        return self.profiles.get(profile_id)

    def get_all_profiles(self):
        """Returns list of all profiles for matching"""
        return list(self.profiles.values())


class Matchmaker:
    """
    Implements matchmaking logic using compatibility rules and weights
    Leverages the knowledge base to find optimal matches
    """
    def _init_(self, knowledge_base, weights=None, max_age_diff=20):
        """
        :param knowledge_base: KnowledgeBase instance
        :param weights: Dict of attribute weights (sum to 1)
        :param max_age_diff: Maximum age difference for normalization
        """
        self.knowledge_base = knowledge_base
        self.weights = weights or {'age': 0.3, 'interests': 0.5, 'location': 0.2}
        self.max_age_diff = max_age_diff  # Used for age difference normalization

    def calculate_compatibility(self, profile_a, profile_b):
        """
        Computes weighted compatibility score between two profiles
        Implements domain-specific matching rules:
        - Age: Normalized inverse age gap
        - Interests: Jaccard similarity of interest sets
        - Location: Exact match check
        """
        # Age Compatibility (Normalized inverse difference)
        age_diff = abs(profile_a['age'] - profile_b['age'])
        age_score = max(0, 1 - (age_diff / self.max_age_diff)) * self.weights['age']

        # Interest Compatibility (Jaccard Index)
        interests_a = set(profile_a['interests'])
        interests_b = set(profile_b['interests'])
        intersection = len(interests_a & interests_b)
        union = len(interests_a | interests_b)
        jaccard = intersection / union if union > 0 else 0
        interest_score = jaccard * self.weights['interests']

        # Location Compatibility (Binary match)
        location_score = (self.weights['location'] 
                          if profile_a['location'] == profile_b['location'] 
                          else 0)

        return age_score + interest_score + location_score

    def find_matches(self, user_id, top_n=5):
        """
        Finds top N matches for a given user based on compatibility scores
        :param user_id: Target user to find matches for
        :param top_n: Number of matches to return
        :return: List of (profile_id, score) tuples sorted by score
        """
        target_profile = self.knowledge_base.get_profile(user_id)
        if not target_profile:
            return []

        matches = []
        for profile in self.knowledge_base.get_all_profiles():
            if profile['id'] == user_id:
                continue  # Skip self-comparison

            score = self.calculate_compatibility(target_profile, profile)
            matches.append((profile['id'], round(score, 2)))

        # Sort by descending score and return top N results
        matches.sort(key=lambda x: x[1], reverse=True)
        return matches[:top_n]


if __name__ == "_main_":
    # Initialize Knowledge Base
    kb = KnowledgeBase()
    
    # Populate with sample profiles
    profiles = [
        {'age': 25, 'interests': ['hiking', 'cooking', 'movies'], 'location': 'New York'},
        {'age': 28, 'interests': ['cooking', 'travel', 'photography'], 'location': 'New York'},
        {'age': 35, 'interests': ['gaming', 'coding', 'reading'], 'location': 'Chicago'},
        {'age': 40, 'interests': ['hiking', 'travel', 'photography'], 'location': 'Boston'},
    ]
    for p in profiles:
        kb.add_profile(p)

    # Initialize Matchmaker with custom weights
    matchmaker = Matchmaker(
        knowledge_base=kb,
        weights={'age': 0.25, 'interests': 0.6, 'location': 0.15},
        max_age_diff=25
    )

    # Find matches for first profile (ID=1)
    print("Top matches for profile 1:")
    for match in matchmaker.find_matches(1, top_n=3):
        print(f"Profile {match[0]}: Compatibility Score {match[1]}")